# Diagnostics/cartes tempêtes à partir de réanalyses ERA5

Calepin à utiliser réaliser les cartes à partir des réanalyses ERA5 permettant d'alimenter le site tempêtes : http://tempetes.canevas.enm.meteo.fr/tempete.py.

- Pour chaque tempête, un fichier de tracking "tempete.txt" obtenu grâce au calepin "ERA5_storms_tracking_auto.ipynb" doit être présent dans le répertoire 'txt/tempete/'


- Pour chaque tempête, les données au format netcdf doivent être téléchargées sur le Copernicus Climate Datastore (CDS) au pas de temps horaire et sur un domaine limité (-100W-50E, 0-90N) :
    - https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=form
    - https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=form


- Les fichiers netcdf suivants doivent être présents dans le dossier data/tempete : 
    - z.nc : géopotentiel
    - v.nc  : tourbillon potentiel
    - q.nc  : humidité spécifique
    - u.nc  : vent zonal
    - v.nc  : vent méridien
    - w.nc  : vitesse verticale
    - t.nc  : température
    
    - msl.nc : pression réduite au niveau de la mer
    - tp.nc  : précipitations totales
    - tcwv.nc  : eau précipitable
    - i10fg.nc  : rafales de vent à 10m
    
Remarque : le calepin permet de gérer la présence de 2 fichiers netcdf si les données sont à cheval sur 2 mois et stockées dans 2 fichiers pour chaque variable (ex : msl1.nc & msl2.nc)

In [ ]:
import os

import xarray as xr
import netCDF4

import numpy as np

from cartopy import config
from cartopy.util import add_cyclic_point
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.path as mpath

from scipy.ndimage import gaussian_filter

import metpy.calc as mpcalc
from metpy.units import units

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def plot_maxmin_points(lon, lat, data, extrema, nsize, symbol, color='k',
                       plotValue=True, transform=None):
    """
    This function will find and plot relative maximum and minimum for a 2D grid. The function
    can be used to plot an H for maximum values (e.g., High pressure) and an L for minimum
    values (e.g., low pressue). It is best to used filetered data to obtain  a synoptic scale
    max/min value. The symbol text can be set to a string value and optionally the color of the
    symbol and any plotted value can be set with the parameter color
    lon = plotting longitude values (2D)
    lat = plotting latitude values (2D)
    data = 2D data that you wish to plot the max/min symbol placement
    extrema = Either a value of max for Maximum Values or min for Minimum Values
    nsize = Size of the grid box to filter the max and min values to plot a reasonable number
    symbol = String to be placed at location of max/min value
    color = String matplotlib colorname to plot the symbol (and numerica value, if plotted)
    plot_value = Boolean (True/False) of whether to plot the numeric value of max/min point
    The max/min symbol will be plotted on the current axes within the bounding frame
    (e.g., clip_on=True)
    """
    from scipy.ndimage import maximum_filter, minimum_filter

    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')

    mxy, mxx = np.where(data_ext == data)

    for i in range(len(mxy)):
        #print('Low '+str(i+1))
        #print(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]], int(data[mxy[i], mxx[i]]))
        ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]], symbol, color=color, size=12,
                clip_on=True, horizontalalignment='center', verticalalignment='center',
                transform=transform)
        ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]],
                '\n' + str(int(data[mxy[i], mxx[i]])),
                color=color, size=10, clip_on=True, fontweight='bold',
                horizontalalignment='center', verticalalignment='top', transform=transform)

def lonflip(da):
    lon_name = 'longitude'
    da['_longitude_adjusted'] = xr.where(
        da[lon_name] > 180,
        da[lon_name] - 360,
        da[lon_name])
    da = (
        da
        .swap_dims({lon_name: '_longitude_adjusted'})
        .sel(**{'_longitude_adjusted': sorted(da._longitude_adjusted)})
        .drop(lon_name))
    da = da.rename({'_longitude_adjusted': lon_name})
    return da

def plot_background(ax):
    ax.coastlines("10m", color='grey', zorder=3)
    ax.gridlines(draw_labels=False, color='gray', alpha=0.8, linestyle='-')
    return ax

def make_boundary_path(lon,lat):
    lons,lats=np.meshgrid(lon,lat)
    boundary_path = np.array([lons[-1,:],lats[-1,:]])
    boundary_path = np.append(boundary_path,np.array([lons[::-1,-1],lats[::-1,-1]]),axis=1)
    boundary_path = np.append(boundary_path,np.array([lons[1,::-1],lats[1,::-1]]),axis=1)
    boundary_path = np.append(boundary_path,np.array([lons[:,1],lats[:,1]]),axis=1)
    boundary_path = mpath.Path(np.swapaxes(boundary_path, 0, 1))
    return boundary_path

def make_animation(png_path, gif_filepath):
    from PIL import Image
    import os
    from IPython.display import Image as IPImage
    from IPython.display import display
    import time
    
    image_folder = png_path # répertoire contenant les fichiers PNG
    output_file = gif_filepath # nom du fichier de sortie
    animation_speed = 0.9 # vitesse de l'animation en secondes
    
    # Liste tous les fichiers PNG dans le répertoire image_folder
    files = sorted(os.listdir(image_folder))
    image_files = [f for f in files if f.endswith('.png')]
    
    # Ouvre chaque fichier PNG et ajoute l'image à une liste
    images = []
    for filename in image_files:
        img = Image.open(os.path.join(image_folder, filename))
        images.append(img)
    
    # Crée l'animation GIF
    images[0].save(output_file, save_all=True, append_images=images[1:], duration=int(animation_speed*1000), loop=0)
    # Affiche l'animation GIF dans Jupyter
    with open(output_file,'rb') as f:
        display(IPImage(data=f.read(), format='png'))
    # Efface les fichiers PNG
    #for filename in image_files:
        #os.remove(image_folder+filename)

In [ ]:
dir_data="E:/BIG_DATA/ERA5_storms/data/"
dir_diag="E:/BIG_DATA/ERA5_storms/diagnostics/"
dir_figs='./figs/'
dir_txt='./txt/'
dir_anim='./anim/'

In [ ]:
#storms=['Nov1982', 'Oct1987', 'Herta', 'Viviane', 'Braer', 'Lothar', 'Martin', 'Klaus', 'Xynthia',
#        'Joachim', 'Zeus', 'Ophelia', 'Eleanor', 'Zorbas', 'Lorenzo', 'Alex', 'Ciaran']
storms=['Alex'] # single storm

In [ ]:
for storm in storms:
    if not os.path.exists(dir_figs+storm):
        os.mkdir(dir_figs+storm)
    if not os.path.exists(dir_anim+storm):
        os.mkdir(dir_anim+storm)
    if not os.path.exists(dir_txt+storm+'.txt'):
        print('Missing tracking file for storm '+storm)

# Concaténation netcdf
<span style="color:red"> Ne faire touner qu'une fois si 2 fichiers netcdf à cheval sur 2 mois, pour concaténer les fichiers netcdf 1 et 2  en 1 seul fichier. Attention à bien vérifier la continuité des données concaténées (souci d'offsset avec la Pmer ?).</span>

In [ ]:
for storm in storms:
    print("Opening netcdf files ...")
    #f1    = xr.open_mfdataset(dir_data+storm+"/msl*.nc") # probleme offset
    f2    = xr.open_mfdataset(dir_data+storm+"/u*.nc")
    f3    = xr.open_mfdataset(dir_data+storm+"/v*.nc")
    f4    = xr.open_mfdataset(dir_data+storm+"/w*.nc")
    f5    = xr.open_mfdataset([dir_data+storm+"/t1.nc",dir_data+storm+"/t2.nc"])
    f6    = xr.open_mfdataset(dir_data+storm+"/pv*.nc")
    f7    = xr.open_mfdataset(dir_data+storm+"/z*.nc")
    f8    = xr.open_mfdataset(dir_data+storm+"/q*.nc")
    f9 = xr.open_mfdataset(dir_data+storm+"/tp*.nc")
    f10 = xr.open_mfdataset(dir_data+storm+"/tcwv*.nc")
    f11= xr.open_mfdataset(dir_data+storm+"/i10fg*.nc")
    
    print("Creating single netcdf files from file1 and file2 ...")    
    f1.to_netcdf(dir_data+storm+'/msl.nc')
    f2.to_netcdf(dir_data+storm+'/u.nc')
    f3.to_netcdf(dir_data+storm+'/v.nc')
    f4.to_netcdf(dir_data+storm+'/w.nc')
    f5.to_netcdf(dir_data+storm+'/t.nc')
    f6.to_netcdf(dir_data+storm+'/pv.nc')
    f7.to_netcdf(dir_data+storm+'/z.nc')
    f8.to_netcdf(dir_data+storm+'/q.nc')
    f9.to_netcdf(dir_data+storm+'/tp.nc')
    f10.to_netcdf(dir_data+storm+'/tcwv.nc')
    f11.to_netcdf(dir_data+storm+'/i10fg.nc')

# Calcul du tourbillon relatif et de la température potentielle équivalente

<span style="color:red"> Ne faire touner qu'une fois pour calculer le toubillon relatif (vort) à partir de u et v et la température potentielle équivalente (thetae) à partir de t et q et sauvegarder les fichiers netcdf vort et thetae dans le dossier diagnostic.</span>

In [ ]:
for storm in storms:
    
    latS=30
    latN=70
    lonW=-60
    lonE=20
    
    if storm=='Ciaran':
        latS=30
        latN=70
        lonW=-70
        lonE=30
        
    if storm=='Lorenzo':
        latS=25
        latN=60
        lonW=-60
        lonE=20
    
    print(storm)
    liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt(dir_txt+storm+'.txt',skiprows=0, dtype='U13,float,float,int',unpack=True)
    date1=str(liste_time[0])
    date2=str(liste_time[-1])
    
    f1    = xr.open_dataset(dir_data+storm+"/u.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS))
    u = f1['u']
    f2    = xr.open_dataset(dir_data+storm+"/v.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS))
    v = f2['v']
    f3    = xr.open_dataset(dir_data+storm+"/t.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS))
    t = f3['t']
    f4    = xr.open_dataset(dir_data+storm+"/q.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS))
    q = f4['q']
    
    time  = u.time.values
    lev  = u.level.values
    lat  = u.latitude.values
    lon  = u.longitude.values

    print('Computing Td ...')
    p=lev[np.newaxis,:,np.newaxis,np.newaxis]
    Td = mpcalc.dewpoint_from_specific_humidity(p* units.hPa, t.values* units.kelvin, q.values* units('kg/kg'))

    print('Computing Thetae ...')
    thetae=mpcalc.equivalent_potential_temperature(p* units.hPa, t, Td)
    thetae=np.array(thetae)-273.15
    #print(thetae)

    print('Computing divergence and vorticity ...')
    omega=7.292e-5
    re=6.37e6
    pi=3.14159265
    
    xlon,ylat=np.meshgrid(lon,lat)
    dlony,dlonx=np.gradient(xlon)
    dlaty,dlatx=np.gradient(ylat)
    dx=re*np.cos(ylat*pi/180)*dlonx*pi/180
    dy=re*dlaty*pi/180
    
    f=2*omega*np.sin(ylat*pi/180)

    ddx_u=np.gradient(u,axis=3)/dx
    ddx_v=np.gradient(v,axis=3)/dx
    ddy_u=np.gradient(u,axis=2)/dy
    ddy_v=np.gradient(v,axis=2)/dy

    div=ddx_u+ddy_v-(v/re)*np.tan(ylat*pi/180)
    vort=ddx_v-ddy_u+(u/re)*np.tan(ylat*pi/180)
    vorta=vort+f
    #print(vort)

    print('Writing vort and thetae netcdf files ...')
    da_vort = xr.DataArray(vort, dims=['time', 'level', 'latitude', 'longitude'],coords={'time': time,'level': lev,'latitude': lat,'longitude': lon})
    da_vort = da_vort.rename("vort")
    print(da_vort)
    da_vort.to_netcdf(dir_diag+storm+'/vort.nc')
    
    da_thetae = xr.DataArray(thetae, dims=['time', 'level', 'latitude', 'longitude'],coords={'time': time,'level': lev,'latitude': lat,'longitude': lon})
    da_thetae = da_thetae.rename("thetae")
    print(da_thetae)
    da_thetae.to_netcdf(dir_diag+storm+'/thetae.nc')

In [ ]:
for storm in storms:
    print("Checking netcdf files ...")
    
    if storm=='Ciaran':
        f0    = xr.open_dataset(dir_data+storm+"/msl1.nc")
        f1    = xr.open_dataset(dir_data+storm+"/msl2.nc")
    else:
        f1    = xr.open_dataset(dir_data+storm+"/msl.nc")
    f2    = xr.open_dataset(dir_data+storm+"/u.nc")
    f3    = xr.open_dataset(dir_data+storm+"/v.nc")
    f4    = xr.open_dataset(dir_data+storm+"/w.nc")
    f5    = xr.open_dataset(dir_data+storm+"/t.nc")
    f6    = xr.open_dataset(dir_data+storm+"/pv.nc")
    f7    = xr.open_dataset(dir_data+storm+"/z.nc")
    f8    = xr.open_dataset(dir_data+storm+"/q.nc")
    f9 = xr.open_dataset(dir_data+storm+"/tp.nc")
    f10 = xr.open_dataset(dir_data+storm+"/tcwv.nc")
    f11= xr.open_dataset(dir_data+storm+"/i10fg.nc")
    f12= xr.open_dataset(dir_diag+storm+"/vort.nc")
    f13= xr.open_dataset(dir_diag+storm+"/thetae.nc")
    
    if storm=='Ciaran':
        print(f0)
    print(f1)
    print(f2)
    print(f3)
    print(f4)
    print(f5)
    print(f6)
    print(f7)
    print(f8)
    print(f9)
    print(f10)
    print(f11)
    print(f12)
    print(f13)

In [ ]:
mslp_levels = np.arange(900,1072,2)
tend_levels=np.arange(-8,8.5,0.5)
vort_levels = np.arange(2,11,1)
th_levels=np.arange(-20,75,5)
w_levels=np.arange(-5.25,0,0.25)
wind_levels = [40, 50, 60, 70, 80, 90, 100, 110, 120]
wind_slice=slice(None,None,10)
pv300_levels=np.arange(1,12.5,0.5)
pv900_levels=np.arange(2,8,0.2)
raf_levels1 = np.arange(25,41,1)
raf_levels2 = np.arange(80,162.5,2.5) # km/h
rr_levels=np.arange(1,6.2,0.2)
z700_levels = np.arange(2000,4000,50)
tcwv_levels=np.arange(2,56,2)
z500_levels = np.arange(4000,6000,50)
t500_levels=np.arange(-50,2.5,2.5)
z15_levels=np.arange(5000,12250,250)

# Altitude géopotentielle à 700 hPa et eau précipitable

In [ ]:
for storm in storms:
    
    latS=30
    latN=70
    lonW=-60
    lonE=20
    
    if storm=='Ciaran':
        latS=30
        latN=70
        lonW=-70
        lonE=30
        
    if storm=='Lorenzo':
        latS=25
        latN=60
        lonW=-60
        lonE=20
        
    bounds = [(lonW, lonE, latS, latN)]
    projection=ccrs.NearsidePerspective(central_longitude=(lonW+lonE)/2, central_latitude=(latS+latN)/2)
    
    liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt(dir_txt+storm+'.txt',skiprows=0, dtype='U13,float,float,int',unpack=True)
    date1=str(liste_time[0])
    date2=str(liste_time[-1])
    
    f1    = xr.open_dataset(dir_data+storm+"/z.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE)).sel(level=700)
    z = f1['z'][:,:,:]/9.81
    f2    = xr.open_dataset(dir_data+storm+"/tcwv.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    tcwv = f2['tcwv'][:,:,:]
    lat  = z.latitude.values
    time  = z.time.values
    lon  = z.longitude.values
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    print(z)
    print(tcwv)

    path=dir_figs+storm+'/Z700_TCWV/'
    if not os.path.exists(path):
        os.mkdir(path)
    
    for i in tqdm(range(len(time))):
        #print(str(time[i])[0:13])
        fig = plt.figure(figsize=(17., 12.))
        ax = fig.add_subplot(1, 1, 1, projection=projection)
        ax.set_title('Storm '+storm+' - Geopotential height at 700 hPa and precipitable water',loc='left',fontsize=14) 		
        ax.set_title(str(time[i])[0:13],loc='right',fontsize=14)
        plot_background(ax)
        ax.set_extent(*bounds, crs=ccrs.PlateCarree())
        boundary_path = make_boundary_path(lon, lat)
        ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())  

        c1 = ax.contour(lon, lat, z[i,:,:], levels=z700_levels, colors="black", linewidths=1, transform=ccrs.PlateCarree())
        ax.clabel(c1,fmt='%4.1i',fontsize=10)
        cf = ax.contourf(lon, lat, tcwv[i,:,:], levels=tcwv_levels, extend='both', cmap='gist_ncar', transform=ccrs.PlateCarree())
        cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
        cb.set_label('mm', size='large')

        #plt.show()
        figname=dir_figs+storm+'/Z700_TCWV/Z700_TCWV_'+str(time[i])[0:13]
        fig.savefig(figname+'.png',bbox_inches='tight')
        plt.close()
        
    gif_filepath = dir_anim+storm+'/Z700_TCWV.gif'
    make_animation(path, gif_filepath)

# Altitude géopotentielle et température à 500 hPa

In [ ]:
for storm in storms:
    
    latS=30
    latN=70
    lonW=-60
    lonE=20
    
    if storm=='Ciaran':
        latS=30
        latN=70
        lonW=-70
        lonE=30
        
    if storm=='Lorenzo':
        latS=25
        latN=60
        lonW=-60
        lonE=20
        
    bounds = [(lonW, lonE, latS, latN)]
    projection=ccrs.NearsidePerspective(central_longitude=(lonW+lonE)/2, central_latitude=(latS+latN)/2)
    
    liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt(dir_txt+storm+'.txt',skiprows=0, dtype='U13,float,float,int',unpack=True)
    date1=str(liste_time[0])
    date2=str(liste_time[-1])

    f1    = xr.open_dataset(dir_data+storm+"/z.nc").sel(time=slice(date1,date2)).sel(level=500).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    z = f1['z'][:,:,:]/9.81
    f2    = xr.open_dataset(dir_data+storm+"/t.nc").sel(time=slice(date1,date2)).sel(level=500).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))    
    t = f2['t'][:,:,:]-273.15

    lat  = z.latitude.values
    time  = z.time.values
    lon  = z.longitude.values
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    print(z)
    print(t)

    path=dir_figs+storm+'/ZT500/'
    if not os.path.exists(path):
        os.mkdir(path)
    
    for i in tqdm(range(len(time))):
        #print(str(time[i])[0:13])
        fig = plt.figure(figsize=(17., 12.))
        ax = fig.add_subplot(1, 1, 1, projection=projection)
        ax.set_title('Storm '+storm+' - Geopotential height and temperature at 500hPa',loc='left',fontsize=14) 		
        ax.set_title(str(time[i])[0:13],loc='right',fontsize=14)
        plot_background(ax)
        ax.set_extent(*bounds, crs=ccrs.PlateCarree())
        boundary_path = make_boundary_path(lon, lat)
        ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())  
        
        c1 = ax.contour(lon, lat, z[i,:,:], levels=z500_levels, colors="black", linewidths=1, transform=ccrs.PlateCarree())
        ax.clabel(c1,fmt='%4.1i',fontsize=10)
        #plot_maxmin_points(lon_grid, lat_grid, z[i,:,:], 'max', 25, symbol='H', color='r',  transform=ccrs.PlateCarree())
        #plot_maxmin_points(lon_grid, lat_grid, z[i,:,:], 'min', 25, symbol='L', color='b', transform=ccrs.PlateCarree())

        cf = ax.contourf(lon, lat, t[i,:,:], levels=t500_levels, extend='both', cmap='jet', transform=ccrs.PlateCarree())
        cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
        cb.set_label('°C', size='large')
        
        #plt.show()
        figname=dir_figs+storm+'/ZT500/ZT500_'+str(time[i])[0:13]
        fig.savefig(figname+'.png',bbox_inches='tight')
        plt.close()
        
    gif_filepath = dir_anim+storm+'/ZT500.gif'
    make_animation(path, gif_filepath)

# Pmer et tracking

In [ ]:
for storm in storms:
    
    latS=30
    latN=70
    lonW=-60
    lonE=20
    
    if storm=='Ciaran':
        latS=30
        latN=70
        lonW=-70
        lonE=30
        
    if storm=='Lorenzo':
        latS=25
        latN=60
        lonW=-60
        lonE=20
        
    bounds = [(lonW, lonE, latS, latN)]
    projection=ccrs.NearsidePerspective(central_longitude=(lonW+lonE)/2, central_latitude=(latS+latN)/2)
    
    liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt(dir_txt+storm+'.txt',skiprows=0, dtype='U13,float,float,int',unpack=True)
    date1=str(liste_time[0])
    date2=str(liste_time[-1])
    
    f1    = xr.open_mfdataset(dir_data+storm+"/msl*.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    mslp = f1['msl'][:,:,:]/100
    lat  = mslp.latitude.values
    lon  = mslp.longitude.values
    time  = mslp.time.values
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    print(mslp)
    
    path=dir_figs+storm+'/MSL_tracking/'
    if not os.path.exists(path):
        os.mkdir(path)
    
    for i in tqdm(range(len(time))):
        
        #print(str(time[i])[0:13])
        fig = plt.figure(figsize=(17., 12.))
        ax = fig.add_subplot(1, 1, 1, projection=projection)
        ax.set_title('Storm '+storm+' - Mean Sea level Pressure and tracking',loc='left',fontsize=14)
        ax.set_title(str(time[i])[0:13],loc='right',fontsize=14)
        
        plot_background(ax)
        ax.set_extent(*bounds, crs=ccrs.PlateCarree())
        boundary_path = make_boundary_path(lon, lat)
        ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())  
        
        c1 = ax.contour(lon, lat, mslp[i,:,:], levels=mslp_levels, colors="black", linewidths=1,
                        transform=ccrs.PlateCarree())
        ax.clabel(c1,fmt='%4.1i',fontsize=10)
        ax.scatter(liste_lon[i],liste_lat[i], c='green', transform=ccrs.PlateCarree())
        ax.plot(liste_lon[0:i+1],liste_lat[0:i+1], c='red', marker='+', transform=ccrs.PlateCarree())
        plot_maxmin_points(lon_grid, lat_grid, mslp[i,:,:], 'min', 25, symbol='L', color='b',
                           transform=ccrs.PlateCarree())
        
        #plt.show()
        figname=dir_figs+storm+'/MSL_tracking/MSL_tracking_'+str(time[i])[0:13]
        fig.savefig(figname+'.png',bbox_inches='tight')
        plt.close()
        
    gif_filepath = dir_anim+storm+'/MSL_tracking.gif'
    make_animation(path, gif_filepath)

# Pmer et tendance de pression

In [ ]:
for storm in storms:
    
    latS=30
    latN=70
    lonW=-60
    lonE=20
    
    if storm=='Ciaran':
        latS=30
        latN=70
        lonW=-70
        lonE=30
        
    if storm=='Lorenzo':
        latS=25
        latN=60
        lonW=-60
        lonE=20

    bounds = [(lonW, lonE, latS, latN)]
    projection=ccrs.NearsidePerspective(central_longitude=(lonW+lonE)/2, central_latitude=(latS+latN)/2)
    
    liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt(dir_txt+storm+'.txt',skiprows=0, dtype='U13,float,float,int',unpack=True)
    date1=str(liste_time[0])
    date2=str(liste_time[-1])
    
    f1    = xr.open_mfdataset(dir_data+storm+"/msl*.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    mslp = f1['msl'][:,:,:]/100
    lat  = mslp.latitude.values
    lon  = mslp.longitude.values
    time  = mslp.time.values
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    print(mslp)
    
    path=dir_figs+storm+'/MSL_Ptend/'
    if not os.path.exists(path):
        os.mkdir(path)
    
    for i in tqdm(range(1,len(time))):
        #print(str(time[i])[0:13])
        
        fig = plt.figure(figsize=(17., 12.))
        ax = fig.add_subplot(1, 1, 1, projection=projection)
        ax.set_title('Storm '+storm+' - Mean Sea level Pressure and pressure tendency',loc='left',fontsize=14)
        ax.set_title(str(time[i])[0:13],loc='right',fontsize=14)
        plot_background(ax)
        ax.set_extent(*bounds, crs=ccrs.PlateCarree())
        boundary_path = make_boundary_path(lon, lat)
        ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())  
        c1 = ax.contour(lon, lat, mslp[i,:,:], levels=mslp_levels, colors="black", linewidths=1,
                        transform=ccrs.PlateCarree())
        ax.clabel(c1,fmt='%4.1i',fontsize=10)
        plot_maxmin_points(lon_grid, lat_grid, mslp[i,:,:], 'min', 25, symbol='L', color='b',
                           transform=ccrs.PlateCarree())
        cf = ax.contourf(lon, lat, mslp[i,:,:]-mslp[i-1,:,:], levels=tend_levels, extend='both', cmap='bwr', transform=ccrs.PlateCarree())
        cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
        cb.set_label('hPa/h', size='large')
        
        #plt.show()
        figname=dir_figs+storm+'/MSL_Ptend/MSL_Ptend_'+str(time[i])[0:13]
        fig.savefig(figname+'.png',bbox_inches='tight')
        plt.close()
    
    gif_filepath = dir_anim+storm+'/MSL_Ptend.gif'
    make_animation(path, gif_filepath)

# Pmer Thetae à 850 hPa et tourbillon relatif à 925 hPa

In [ ]:
for storm in storms:
    
    latS=30
    latN=70
    lonW=-60
    lonE=20
    
    if storm=='Ciaran':
        latS=30
        latN=70
        lonW=-70
        lonE=30
        
    if storm=='Lorenzo':
        latS=25
        latN=60
        lonW=-60
        lonE=20
        
    bounds = [(lonW, lonE, latS, latN)]
    projection=ccrs.NearsidePerspective(central_longitude=(lonW+lonE)/2, central_latitude=(latS+latN)/2)
    
    liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt(dir_txt+storm+'.txt',skiprows=0, dtype='U13,float,float,int',unpack=True)
    date1=str(liste_time[0])
    date2=str(liste_time[-1])
    
    f0    = xr.open_mfdataset(dir_data+storm+"/msl*.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    mslp = f0['msl'][:,:,:]/100
    f1    = xr.open_dataset(dir_diag+storm+"/thetae.nc").sel(level=850).sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    thetae = f1['thetae'][:,:,:]
    f2    = xr.open_dataset(dir_diag+storm+"/vort.nc").sel(level=925).sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    vort = f2['vort'][:,:,:]
    time  = mslp.time.values
    lev  = vort.level.values
    lat  = mslp.latitude.values
    lon  = mslp.longitude.values
    lon_grid, lat_grid = np.meshgrid(lon, lat) # 2D lat/lon for maxmin function
    print(mslp)
    print(thetae)
    print(vort)
    
    path=(dir_figs+storm+'/MSL_Thetae850_vort925/')
    if not os.path.exists(path):
        os.mkdir(path)
    
    for i in tqdm(range(len(time))):
        
        #print(str(time[i])[0:13])
        fig = plt.figure(figsize=(17., 12.))
        ax = fig.add_subplot(1, 1, 1, projection=projection)
        ax.set_title('Storm '+storm+' - MSLP, '+r"$\theta_e$ "+'at 850 hPa and relative vorticity at 925 hPa',loc='left',fontsize=14)
        ax.set_title(str(time[i])[0:13],loc='right',fontsize=14)
        plot_background(ax)
        ax.set_extent(*bounds, crs=ccrs.PlateCarree())
        boundary_path = make_boundary_path(lon, lat)
        ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())  
        c1 = ax.contour(lon, lat, mslp[i,:,:], levels=mslp_levels, colors="brown", linewidths=1,
                        transform=ccrs.PlateCarree())
        #ax.clabel(c1,fmt='%4.1i',fontsize=10)
        plot_maxmin_points(lon_grid, lat_grid, mslp[i,:,:], 'min', 25, symbol='L', color='b',
                           transform=ccrs.PlateCarree())
        c1 = ax.contour(lon, lat, vort[i,:,:]*1e04, levels=vort_levels[vort_levels != 0], colors="black",
                        linewidths=1, transform=ccrs.PlateCarree())
        ax.clabel(c1,fmt='%2.1i',fontsize=10)	

        cf = ax.contourf(lon, lat, thetae[i,:,:], levels=th_levels, extend='both', cmap='jet', transform=ccrs.PlateCarree())
        cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
        cb.set_label('°C', size='large')
        
        #plt.show()
        figname=path+'MSL_Thetae850_vort925_'+str(time[i])[0:13]
        fig.savefig(figname+'.png',bbox_inches='tight')
        plt.close()
    
    gif_filepath = dir_anim+storm+'/MSL_Thetae850_vort925.gif'
    make_animation(path, gif_filepath)

# Pmer, module du vent à 300 hPa et ascendance à 600 hPa

In [ ]:
for storm in storms:
    
    latS=30
    latN=70
    lonW=-60
    lonE=20
    
    if storm=='Ciaran':
        latS=30
        latN=70
        lonW=-70
        lonE=30
        
    if storm=='Lorenzo':
        latS=25
        latN=60
        lonW=-60
        lonE=20
        
    bounds = [(lonW, lonE, latS, latN)]
    projection=ccrs.NearsidePerspective(central_longitude=(lonW+lonE)/2, central_latitude=(latS+latN)/2)
    
    liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt(dir_txt+storm+'.txt',skiprows=0, dtype='U13,float,float,int',unpack=True)
    date1=str(liste_time[0])
    date2=str(liste_time[-1])
    
    f1    = xr.open_mfdataset(dir_data+storm+"/msl*.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    mslp = f1['msl'][:,:,:]/100
    f2    = xr.open_dataset(dir_data+storm+"/u.nc").sel(time=slice(date1,date2)).sel(level=300).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    u = f2['u'][:,:,:]
    f3    = xr.open_dataset(dir_data+storm+"/v.nc").sel(time=slice(date1,date2)).sel(level=300).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    v = f3['v'][:,:,:]
    f4    = xr.open_dataset(dir_data+storm+"/w.nc").sel(time=slice(date1,date2)).sel(level=600).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    w = f4['w'][:,:,:]
    
    lat  = mslp.latitude.values
    time  = mslp.time.values
    lon  = u.longitude.values
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    print(mslp)
    print(u)
    print(v)
    
    uv_mod=(u**2+v**2)**0.5
    print(uv_mod)
    w_filter = gaussian_filter(w, sigma=2.0)
    
    
    path=dir_figs+storm+'/MSL_FF300_VV600/'
    if not os.path.exists(path):
        os.mkdir(path)
    
    for i in tqdm(range(len(time))):
        
        #print(str(time[i])[0:13])
        fig = plt.figure(figsize=(17., 12.))
        ax = fig.add_subplot(1, 1, 1, projection=projection)
        ax.set_title('Storm '+storm+' - MSLP, horizontal wind module at 300 hPa and ascending VV at 600 hPa',loc='left',fontsize=14)
        ax.set_title(str(time[i])[0:13],loc='right',fontsize=14)
        plot_background(ax)
        ax.set_extent(*bounds, crs=ccrs.PlateCarree())
        boundary_path = make_boundary_path(lon, lat)
        ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())  
        
        c1 = ax.contour(lon, lat, mslp[i,:,:], levels=mslp_levels, colors="black", linewidths=1,
                        transform=ccrs.PlateCarree(), zorder=20)
        ax.clabel(c1,fmt='%4.1i',fontsize=10)
        plot_maxmin_points(lon_grid, lat_grid, mslp[i,:,:], 'min', 25, symbol='L', color='b', transform=ccrs.PlateCarree())
        
        cf = ax.contourf(lon, lat, uv_mod[i,:,:], levels=wind_levels, extend='max', cmap='Spectral_r',
                         transform=ccrs.PlateCarree())
        cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
        cb.set_label('m s$^{-1}$', size='large')
        
        # Add the wind barbs
        #ax.barbs(lon[wind_slice],lat[wind_slice],u[i,wind_slice,wind_slice],v[i,wind_slice,wind_slice]
        #,pivot='middle',color='r',length=6.5,transform=ccrs.PlateCarree())
        
        # Vertical velocity at 600 hPa (ascending) in contours
        #c2=ax.contour(lon,lat,w[i,:,:],w_levels[w_levels!=0],colors='brown',linewidths=1,transform=ccrs.PlateCarree())
        c2=ax.contour(lon,lat,w_filter[i,:,:],w_levels[w_levels!=0],colors='brown',linewidths=1,
                      transform=ccrs.PlateCarree())
        #plt.show()
        figname=path+'MSL_FF300_VV600_'+str(time[i])[0:13]
        fig.savefig(figname+'.png',bbox_inches='tight')
        plt.close()
        
    gif_filepath = dir_anim+storm+'/MSL_FF300_VV600.gif'
    make_animation(path, gif_filepath)

# Pmer et tourbillon potentiel à 300 hPa

In [ ]:
for storm in storms:
    
    latS=30
    latN=70
    lonW=-60
    lonE=20
    
    if storm=='Ciaran':
        latS=30
        latN=70
        lonW=-70
        lonE=30
        
    if storm=='Lorenzo':
        latS=25
        latN=60
        lonW=-60
        lonE=20
        
    bounds = [(lonW, lonE, latS, latN)]
    projection=ccrs.NearsidePerspective(central_longitude=(lonW+lonE)/2, central_latitude=(latS+latN)/2)
    
    liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt(dir_txt+storm+'.txt',skiprows=0, dtype='U13,float,float,int',unpack=True)
    date1=str(liste_time[0])
    date2=str(liste_time[-1])                            
    
    f1    = xr.open_mfdataset(dir_data+storm+"/msl*.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    mslp = f1['msl'][:,:,:]/100
    f2    = xr.open_dataset(dir_data+storm+"/pv.nc").sel(level=300).sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    pv = f2['pv'][:,:,:]* 1e06
    
    lat  = mslp.latitude.values
    time  = mslp.time.values
    lon  = mslp.longitude.values
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    print(mslp)
    print(pv)
    
    path=dir_figs+storm+'/MSL_PV300/'
    if not os.path.exists(path):
        os.mkdir(path)
        
    for i in tqdm(range(len(time))):
        #print(str(time[i])[0:13])
        
        fig = plt.figure(figsize=(17., 12.))
        ax = fig.add_subplot(1, 1, 1, projection=projection)
        ax.set_title('Storm '+storm+ '- MSLP and potential vorticity at 300 hPa',loc='left',fontsize=14)
        ax.set_title(str(time[i])[0:13],loc='right',fontsize=14)
        plot_background(ax)
        ax.set_extent(*bounds, crs=ccrs.PlateCarree())
        boundary_path = make_boundary_path(lon, lat)
        ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())  

        c1 = ax.contour(lon, lat, mslp[i,:,:], levels=mslp_levels, colors="black", linewidths=1,
                        transform=ccrs.PlateCarree(), zorder=20)
        ax.clabel(c1,fmt='%4.1i',fontsize=10)
        plot_maxmin_points(lon_grid, lat_grid, mslp[i,:,:], 'min', 25, symbol='L', color='b',
                           transform=ccrs.PlateCarree())
        cf = ax.contourf(lon, lat, pv[i,:,:], levels=pv300_levels, extend='max', cmap='viridis', transform=ccrs.PlateCarree())
        cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
        cb.set_label('PVU', size='large')
        
        #plt.show()
        figname=dir_figs+storm+'/MSL_PV300/MSL_PV300_'+str(time[i])[0:13]
        fig.savefig(figname+'.png',bbox_inches='tight')
        plt.close()
        
    gif_filepath = dir_anim+storm+'/MSL_PV300.gif'
    make_animation(path, gif_filepath)

# Pmer et tourbillon potentiel à 900 hPa

In [ ]:
for storm in storms:
    
    latS=30
    latN=70
    lonW=-60
    lonE=20
    
    if storm=='Ciaran':
        latS=30
        latN=70
        lonW=-70
        lonE=30
        
    if storm=='Lorenzo':
        latS=25
        latN=60
        lonW=-60
        lonE=20
        
    bounds = [(lonW, lonE, latS, latN)]
    projection=ccrs.NearsidePerspective(central_longitude=(lonW+lonE)/2, central_latitude=(latS+latN)/2)
    
    liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt(dir_txt+storm+'.txt',skiprows=0, dtype='U13,float,float,int',unpack=True)
    date1=str(liste_time[0])
    date2=str(liste_time[-1])
    
    f1    = xr.open_mfdataset(dir_data+storm+"/msl*.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    mslp = f1['msl'][:,:,:]/100
    f2    = xr.open_dataset(dir_data+storm+"/pv.nc").sel(level=900).sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    pv = f2['pv'][:,:,:]* 1e06
    
    lat  = mslp.latitude.values
    time  = mslp.time.values
    lon  = mslp.longitude.values
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    print(mslp)
    print(pv)
    
    path=dir_figs+storm+'/MSL_PV900/'
    if not os.path.exists(path):
        os.mkdir(path)
    
    for i in tqdm(range(len(time))):
        #print(str(time[i])[0:13])
        fig = plt.figure(figsize=(17., 12.))
        ax = fig.add_subplot(1, 1, 1, projection=projection)
        ax.set_title('Storm '+storm+' - MSLP and potential vorticity at 900 hPa',loc='left',fontsize=14)
        ax.set_title(str(time[i])[0:13],loc='right',fontsize=14)
        plot_background(ax)
        ax.set_extent(*bounds, crs=ccrs.PlateCarree())
        boundary_path = make_boundary_path(lon, lat)
        ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())  
        
        c1 = ax.contour(lon, lat, mslp[i,:,:], levels=mslp_levels, colors="black", linewidths=1, transform=ccrs.PlateCarree())
        ax.clabel(c1,fmt='%4.1i',fontsize=10)
        plot_maxmin_points(lon_grid, lat_grid, mslp[i,:,:], 'min', 25, symbol='L', color='b', transform=ccrs.PlateCarree())
        
        cf = ax.contourf(lon, lat, pv[i,:,:], levels=pv900_levels, extend='max', cmap='viridis', transform=ccrs.PlateCarree())
        cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
        cb.set_label('PVU', size='large')
        
        #plt.show()
        figname=dir_figs+storm+'/MSL_PV900/MSL_PV900_'+str(time[i])[0:13]
        fig.savefig(figname+'.png',bbox_inches='tight')
        plt.close()
        
    gif_filepath = dir_anim+storm+'/MSL_PV900.gif'
    make_animation(path, gif_filepath)

# Pmer et rafales de vent à 10m (grand domaine)

In [ ]:
for storm in storms:
    
    latS=30
    latN=70
    lonW=-60
    lonE=20
    
    if storm=='Ciaran':
        latS=30
        latN=70
        lonW=-70
        lonE=30
        
    if storm=='Lorenzo':
        latS=25
        latN=60
        lonW=-60
        lonE=20
        
    bounds = [(lonW, lonE, latS, latN)]
    projection=ccrs.NearsidePerspective(central_longitude=(lonW+lonE)/2, central_latitude=(latS+latN)/2)
    
    liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt(dir_txt+storm+'.txt',skiprows=0, dtype='U13,float,float,int',unpack=True)
    date1=str(liste_time[0])
    date2=str(liste_time[-1])
    
    f1    = xr.open_mfdataset(dir_data+storm+"/msl*.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    mslp = f1['msl'][:,:,:]/100
    f2    = xr.open_dataset(dir_data+storm+"/i10fg.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    raf = f2['i10fg'][:,:,:]
    
    lat  = mslp.latitude.values
    time  = mslp.time.values
    lon  = mslp.longitude.values
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    print(mslp)
    print(raf)
    
    path=dir_figs+storm+'/MSL_FF10m/'
    if not os.path.exists(path):
        os.mkdir(path)
    
    for i in tqdm(range(len(time))): 
        #print(str(time[i])[0:13])
        fig = plt.figure(figsize=(17., 12.))
        ax = fig.add_subplot(1, 1, 1, projection=projection)
        ax.set_title('Storm '+storm+' - MSLP and 10 meters instantaneous wind gust (m/s)',loc='left',fontsize=14)
        ax.set_title(str(time[i])[0:13],loc='right',fontsize=14)
        plot_background(ax)
        ax.set_extent(*bounds, crs=ccrs.PlateCarree())
        boundary_path = make_boundary_path(lon, lat)
        ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())  
        
        c1 = ax.contour(lon, lat, mslp[i,:,:], levels=mslp_levels, colors="black", linewidths=1, transform=ccrs.PlateCarree())
        ax.clabel(c1,fmt='%4.1i',fontsize=10)
        plot_maxmin_points(lon_grid, lat_grid, mslp[i,:,:], 'min', 25, symbol='L', color='b', transform=ccrs.PlateCarree())
        
        cf = ax.contourf(lon, lat, raf[i,:,:], levels=raf_levels1, extend='max', cmap='jet', transform=ccrs.PlateCarree())
        cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
        cb.set_label('m/s', size='large')
        
        #plt.show()
        figname=dir_figs+storm+'/MSL_FF10m/MSL_FF10m_big_'+str(time[i])[0:13]
        fig.savefig(figname+'.png',bbox_inches='tight')
        plt.close()
        
    gif_filepath = dir_anim+storm+'/MSL_FF10m_big.gif'
    make_animation(path, gif_filepath)

# Pmer et rafales de vent à 10m (domaine France)

In [ ]:
lon1=-10
lon2=15
lat1=41
lat2=53
projection2=ccrs.AlbersEqualArea(central_longitude=(lon1+lon2)/2, central_latitude=(lat1+lat2)/2)

resol = '50m'
bodr = cfeature.NaturalEarthFeature(category='cultural', 
                                           name='admin_0_boundary_lines_land', scale=resol, facecolor='none', alpha=0.7)
rivers = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', 
                                             scale=resol, edgecolor='b', facecolor='none')

for storm in storms:
    
    liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt(dir_txt+storm+'.txt',skiprows=0, dtype='U13,float,float,int',unpack=True)
    date1=str(liste_time[0])
    date2=str(liste_time[-1])

    f1    = xr.open_mfdataset(dir_data+storm+"/msl*.nc").sel(time=slice(date1,date2)).sel(latitude=slice(lat2,lat1)).sel(longitude=slice(lon1,lon2))
    mslp = f1['msl'][:,:,:]/100
    f2    = xr.open_dataset(dir_data+storm+"/i10fg.nc").sel(time=slice(date1,date2)).sel(latitude=slice(lat2,lat1)).sel(longitude=slice(lon1,lon2))
    raf = f2['i10fg'][:,:,:]*3.6
    f3    = xr.open_dataset(dir_diag+storm+"/thetae.nc").sel(level=850).sel(time=slice(date1,date2)).sel(latitude=slice(lat2,lat1)).sel(longitude=slice(lon1,lon2))
    thetae = f3['thetae'][:,:,:]
    
    lat  = mslp.latitude.values
    time  = mslp.time.values
    lon  = mslp.longitude.values
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    print(mslp)
    print(raf)

    path=dir_figs+storm+'/MSL_FF10m_zoom/'
    if not os.path.exists(path):
        os.mkdir(path)
    
    for i in tqdm(range(len(time))):
        #print(str(time[i])[0:13])
        #print('max wind gust')
        max_gust=int(np.amax(raf[i,:,:]))
        maxloc=np.where(raf == np.amax(raf[i,:,:]))
        for cord in list(zip(maxloc[1],maxloc[2])):
            latmax=cord[0]
            lonmax=cord[1]
        #print('max wind gust latitude')
        #print(lat[latmax])
        #print('max wind gust longitude')
        #print(lon[lonmax])
        fig = plt.figure(figsize=(17., 12.))
        fig.suptitle('Storm '+storm+' - MSLP, '+r"$\theta_e$ "+'at 850 hPa and 10 meters instantaneous wind gust (km/h)',fontsize=16)

        ax = fig.add_subplot(1, 1, 1, projection=projection2)
        ax.set_title(str(time[i])[0:13],loc='center',fontsize=14)
        ax.set_title('max gust :'+str(max_gust)+'km/h',loc='right',fontsize=14)
        
        ax.coastlines("10m", color='grey', zorder=3)
        ax.add_feature(rivers, edgecolor='k', linewidth=0.5)
        ax.add_feature(bodr, linestyle='--', edgecolor='grey', alpha=1)
        
        bounds2 = [(lon1, lon2, lat1, lat2)]
        ax.set_extent(*bounds2, crs=ccrs.PlateCarree())
        
        boundary_path = make_boundary_path(lon, lat)
        ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())  

        c1 = ax.contour(lon, lat, mslp[i,:,:], levels=mslp_levels, colors="black", linewidths=1, transform=ccrs.PlateCarree())
        ax.clabel(c1,fmt='%4.1i',fontsize=10)
        plot_maxmin_points(lon_grid, lat_grid, mslp[i,:,:], 'min', 25, symbol='L', color='b', transform=ccrs.PlateCarree())

        c2 = ax.contour(lon, lat, thetae[i,:,:], levels=th_levels, colors="brown", linewidths=1,
                        transform=ccrs.PlateCarree())
        ax.clabel(c2,fmt='%2.1i',fontsize=10)
                
        cf = ax.contourf(lon, lat, raf[i,:,:], levels=raf_levels2, extend='max', cmap='jet', transform=ccrs.PlateCarree())
        cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.15, extendrect='True')
        cb.set_label('km/h', size='large')
        
        ax.scatter(lon[lonmax], lat[latmax], color='green',transform=ccrs.PlateCarree())
        
        #plt.show()
        figname=dir_figs+storm+'/MSL_FF10m_zoom/MSL_FF10m_zoom_'+str(time[i])[0:13]
        fig.savefig(figname+'.png',bbox_inches='tight')
        plt.close()
        
    gif_filepath = dir_anim+storm+'/MSL_FF10m_zoom.gif'
    make_animation(path, gif_filepath)

# Pmer et précipitations totales

In [ ]:
for storm in storms:
    
    latS=30
    latN=70
    lonW=-60
    lonE=20
    
    if storm=='Ciaran':
        latS=30
        latN=70
        lonW=-70
        lonE=30
        
    if storm=='Lorenzo':
        latS=25
        latN=60
        lonW=-60
        lonE=20
        
    bounds = [(lonW, lonE, latS, latN)]
    projection=ccrs.NearsidePerspective(central_longitude=(lonW+lonE)/2, central_latitude=(latS+latN)/2)
    
    liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt(dir_txt+storm+'.txt',skiprows=0, dtype='U13,float,float,int',unpack=True)
    date1=str(liste_time[0])
    date2=str(liste_time[-1])
    
    f1    = xr.open_mfdataset(dir_data+storm+"/msl*.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    mslp = f1['msl'][:,:,:]/100
    f2    = xr.open_dataset(dir_data+storm+"/tp.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    rr = f2['tp'][:,:,:]*1000
    
    lat  = mslp.latitude.values
    time  = mslp.time.values
    lon  = mslp.longitude.values
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    print(mslp)
    print(rr)

    path=dir_figs+storm+'/MSL_RR/'
    if not os.path.exists(path):
        os.mkdir(path)
        
    for i in tqdm(range(len(time))):
        #print(str(time[i])[0:13])
        fig = plt.figure(figsize=(17., 12.))
        ax = fig.add_subplot(1, 1, 1, projection=projection)
        ax.set_title('Storm '+storm+' - MSLP and 1h accumulated rainfall',loc='left',fontsize=14)
        ax.set_title(str(time[i])[0:13],loc='right',fontsize=14)
        plot_background(ax)
        ax.set_extent(*bounds, crs=ccrs.PlateCarree())
        boundary_path = make_boundary_path(lon, lat)
        ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())  
                  
        c1 = ax.contour(lon, lat, mslp[i,:,:], levels=mslp_levels, colors="black", linewidths=1,
                        transform=ccrs.PlateCarree())
        ax.clabel(c1,fmt='%4.1i',fontsize=10)
        plot_maxmin_points(lon_grid, lat_grid, mslp[i,:,:], 'min', 25, symbol='L', color='b',
                           transform=ccrs.PlateCarree())
        cf = ax.contourf(lon, lat, rr[i,:,:], levels=rr_levels, extend='max', cmap='cool',
                         transform=ccrs.PlateCarree())
        cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
        cb.set_label('mm', size='large')
                  
        #plt.show()
        figname=dir_figs+storm+'/MSL_RR/MSL_RR_'+str(time[i])[0:13]
        fig.savefig(figname+'.png',bbox_inches='tight')
        plt.close()
        
    gif_filepath = dir_anim+storm+'/MSL_RR.gif'
    make_animation(path, gif_filepath)

# Tourbillon potentiel à 300 hPa, thetae à 850 hPa et tourbillon relatif à 925 hPa

In [ ]:
for storm in storms:
    
    latS=30
    latN=70
    lonW=-60
    lonE=20
    
    if storm=='Ciaran':
        latS=30
        latN=70
        lonW=-70
        lonE=30
        
    if storm=='Lorenzo':
        latS=25
        latN=60
        lonW=-60
        lonE=20
        
    bounds = [(lonW, lonE, latS, latN)]
    projection=ccrs.NearsidePerspective(central_longitude=(lonW+lonE)/2, central_latitude=(latS+latN)/2)
    
    liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt(dir_txt+storm+'.txt',skiprows=0, dtype='U13,float,float,int',unpack=True)
    date1=str(liste_time[0])
    date2=str(liste_time[-1])
    
    f0    = xr.open_dataset(dir_data+storm+"/pv.nc").sel(level=300).sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    pv = f0['pv'][:,:,:]* 1e06
    f1    = xr.open_dataset(dir_diag+storm+"/thetae.nc").sel(level=850).sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    thetae = f1['thetae'][:,:,:]
    f2    = xr.open_dataset(dir_diag+storm+"/vort.nc").sel(level=925).sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    vort = f2['vort'][:,:,:]
    
    time  = pv.time.values
    lev  = vort.level.values
    lat  = pv.latitude.values
    lon  = pv.longitude.values
    lon_grid, lat_grid = np.meshgrid(lon, lat) # 2D lat/lon for maxmin function
    print(pv)
    print(thetae)
    print(vort)

    path=(dir_figs+storm+'/PV300_Thetae850_vort925/')                          
    if not os.path.exists(path):
        os.mkdir(path)

    for i in tqdm(range(len(time))):
        #print(str(time[i])[0:13])
        fig = plt.figure(figsize=(17., 12.))
        ax = fig.add_subplot(1, 1, 1, projection=projection)
        ax.set_title('Storm '+storm+' - PV at 300 hPa, '+r"$\theta_e$ "+'at 850 hPa and relative vorticity at 925 hPa',loc='left',fontsize=14)
        ax.set_title(str(time[i])[0:13],loc='right',fontsize=14)

        plot_background(ax)
        ax.set_extent(*bounds, crs=ccrs.PlateCarree())
        boundary_path = make_boundary_path(lon, lat)
        ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())  

        c1 = ax.contour(lon, lat, pv[i,:,:], levels=pv300_levels, colors="brown", linewidths=1, transform=ccrs.PlateCarree())
        ax.clabel(c1,fmt='%2.1f',fontsize=10)
        
        c1 = ax.contour(lon, lat, vort[i,:,:]*1e04, levels=vort_levels[vort_levels != 0], colors="black", linewidths=1,
                        transform=ccrs.PlateCarree())
        ax.clabel(c1,fmt='%2.1i',fontsize=10)
        
        cf = ax.contourf(lon, lat, thetae[i,:,:], levels=th_levels, extend='both', cmap='jet', transform=ccrs.PlateCarree())
        cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
        cb.set_label('°C', size='large')
        
        #plt.show()
        figname=path+'PV300_Thetae850_vort925_'+str(time[i])[0:13]
        fig.savefig(figname+'.png',bbox_inches='tight')
        plt.close()
        
    gif_filepath = dir_anim+storm+'/PV300_Thetae850_vort925.gif'
    make_animation(path, gif_filepath)

# Tourbillon potentiel sur la surface 315K, thetae à 850 hPa et tourbillon relatif à 925 hPa

In [ ]:
isen_level=[315.]* units.kelvin

for storm in storms:
    
    latS=30
    latN=70
    lonW=-60
    lonE=20
    
    if storm=='Ciaran':
        latS=30
        latN=70
        lonW=-70
        lonE=30
        
    if storm=='Lorenzo':
        latS=25
        latN=60
        lonW=-60
        lonE=20
        
    bounds = [(lonW, lonE, latS, latN)]
    projection=ccrs.NearsidePerspective(central_longitude=(lonW+lonE)/2, central_latitude=(latS+latN)/2)
    
    liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt(dir_txt+storm+'.txt',skiprows=0, dtype='U13,float,float,int',unpack=True)
    date1=str(liste_time[0])
    date2=str(liste_time[-1])
    
    f1    = xr.open_dataset(dir_data+storm+"/u.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    u = f1['u']
    f2    = xr.open_dataset(dir_data+storm+"/v.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    v = f2['v']
    f3    = xr.open_dataset(dir_data+storm+"/t.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    t = f3['t']
    f4    = xr.open_dataset(dir_data+storm+"/pv.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    pv = f4['pv']* 1e06
    f5    = xr.open_dataset(dir_diag+storm+"/thetae.nc").sel(level=850).sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    thetae = f5['thetae'][:,:,:]
    f6    = xr.open_dataset(dir_diag+storm+"/vort.nc").sel(level=925).sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    vort = f6['vort'][:,:,:]
    
    time  = u.time.values
    lev  = u.level.values
    lat  = u.latitude.values
    lon  = u.longitude.values
    print(u)
    print(t)
 
    print('Computing isentropic PV ...')
    isen_press, isen_u, isen_v, isen_pv = mpcalc.isentropic_interpolation(isen_level, lev* units.hPa, t, u, v, pv)
    isen_press = isen_press.squeeze()
    isen_u = isen_u.squeeze()
    isen_v = isen_v.squeeze()
    isen_pv = isen_pv.squeeze()
    
    print(isen_pv.shape)
    
    path=(dir_figs+storm+'/PV315K_Thetae850_vort925/')                          
    if not os.path.exists(path):
        os.mkdir(path)

    for i in tqdm(range(len(time))):
        #print(str(time[i])[0:13])
        fig = plt.figure(figsize=(17., 12.))
        ax = fig.add_subplot(1, 1, 1, projection=projection)
        ax.set_title('Storm '+storm+' - PV at 315K, '+r"$\theta_e$ "+'at 850 hPa and relative vorticity at 925 hPa',loc='left',fontsize=14)
        ax.set_title(str(time[i])[0:13],loc='right',fontsize=14)

        plot_background(ax)
        ax.set_extent(*bounds, crs=ccrs.PlateCarree())
        boundary_path = make_boundary_path(lon, lat)
        ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())  

        c1 = ax.contour(lon, lat, isen_pv[i,:,:], levels=pv300_levels, colors="brown", linewidths=1, transform=ccrs.PlateCarree())
        ax.clabel(c1,fmt='%2.1f',fontsize=10)
        
        c1 = ax.contour(lon, lat, vort[i,:,:]*1e04, levels=vort_levels[vort_levels != 0], colors="black", linewidths=1,
                        transform=ccrs.PlateCarree())
        ax.clabel(c1,fmt='%2.1i',fontsize=10)
        
        cf = ax.contourf(lon, lat, thetae[i,:,:], levels=th_levels, extend='both', cmap='jet', transform=ccrs.PlateCarree())
        cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
        cb.set_label('°C', size='large')
        
        #plt.show()
        figname=path+'PV315_Thetae850_vort925_'+str(time[i])[0:13]
        fig.savefig(figname+'.png',bbox_inches='tight')
        plt.close()
        
    gif_filepath = dir_anim+storm+'/PV315K_Thetae850_vort925.gif'
    make_animation(path, gif_filepath)

# Z1.5PVU, thetae à 850 hPa et tourbillon relatif à 925 hPa

In [ ]:
pvlev=1.5
    
for storm in storms:
    
    latS=30
    latN=70
    lonW=-60
    lonE=20
    
    if storm=='Ciaran':
        latS=30
        latN=70
        lonW=-70
        lonE=30
        
    if storm=='Lorenzo':
        latS=25
        latN=60
        lonW=-60
        lonE=20
        
    bounds = [(lonW, lonE, latS, latN)]
    projection=ccrs.NearsidePerspective(central_longitude=(lonW+lonE)/2, central_latitude=(latS+latN)/2)
    
    liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt(dir_txt+storm+'.txt',skiprows=0, dtype='U13,float,float,int',unpack=True)
    date1=str(liste_time[0])
    date2=str(liste_time[-1])
    
    f0    = xr.open_dataset(dir_data+storm+"/pv.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    pv = f0['pv'][:,:,:]* 1e06
    f1    = xr.open_dataset(dir_data+storm+"/u.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    u = f1['u'][:,:,:,:]
    f2    = xr.open_dataset(dir_data+storm+"/v.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    v = f2['v'][:,:,:,:]
    f5    = xr.open_dataset(dir_data+storm+"/z.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    z = f5['z'][:,:,:,:]/9.81
    f6    = xr.open_dataset(dir_diag+storm+"/thetae.nc").sel(level=850).sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    thetae = f6['thetae'][:,:,:]
    f7    = xr.open_dataset(dir_diag+storm+"/vort.nc").sel(level=925).sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
    vort = f7['vort'][:,:,:]
            
    time  = u.time.values
    lev  = u.level.values
    lat  = u.latitude.values
    lon  = u.longitude.values
    lon_grid, lat_grid = np.meshgrid(lon, lat) # 2D lat/lon for maxmin function
    print(u)
    print(t)
    print(pv)
    
    altpvu=[]
    u_pvu=[]
    v_pvu=[]
    z_pvu=[]
    ff_pvu=[]
    
    for k in tqdm(range(0,len(time))):
        #print(' Computing Z 1.5 PVU ...')
        #print(str(time[k])[0:13])
        PV=pv[k,:,:,:]
        U=u[k,:,:,:]
        V=v[k,:,:,:]
        Z=z[k,:,:,:]

        pv1=np.array(PV)
        u1=np.array(U)
        v1=np.array(V)
        z1=np.array(Z)
        pv2=pv1[:-10,:,:]  #from 100 to 750 hPa

        ppvu_arg_sup=np.argwhere((pv2[:-1,:,:]>=pvlev)   & (pv2[1:,:,:]<pvlev))
        ppvu_arg_inf=ppvu_arg_sup.copy()
        ppvu_arg_inf[:,0]=(ppvu_arg_sup[:,0]+1)

        pvu_sup = pv1[ppvu_arg_sup[:,0],ppvu_arg_sup[:,1],ppvu_arg_sup[:,2]]
        pvu_inf = pv1[ppvu_arg_inf[:,0],ppvu_arg_inf[:,1],ppvu_arg_inf[:,2]]

        ppvu = np.zeros((len(lat),len(lon)))
        u_pvu=np.zeros((len(lat),len(lon)))
        v_pvu=np.zeros((len(lat),len(lon)))
        ff_pvu=np.zeros((len(lat),len(lon)))
        
        ppvu[ppvu_arg_sup[:,1],ppvu_arg_sup[:,2]] = (lev[ppvu_arg_sup[:,0]])*(pvlev-pvu_inf[:])/(pvu_sup[:]-pvu_inf[:])+lev[ppvu_arg_inf[:,0]]*(pvu_sup[:]-pvlev)/(pvu_sup[:]-pvu_inf[:])#pv[ppvu_arg_inf[:,0],ppvu_arg_inf[:,1],ppvu_arg_inf[:,2]])/(pv[ppvu_arg_sup[:,0],ppvu_arg_sup[:,1],ppvu_arg_sup[:,2]]-pv[ppvu_arg_inf[:,0],ppvu_arg_inf[:,1],ppvu_arg_inf[:,2]])+level[ppvu_arg_inf[:]]*(pv[ppvu_arg_sup[:,0],ppvu_arg_sup[:,1],ppvu_arg_sup[0,2]]-pvlev)/(pvppvu_arg_sup[:,0],ppvu_arg_sup[:,1],ppvu_arg_sup[:,2])-pv[ppvu_arg_inf[:,0],ppvu_arg_inf[:,1],ppvu_arg_inf[0,2])#(level[ppvu_arg_sup[:,0]]/(pvu_sup-pvu_inf))*(pvlev-pvu_inf[0])+(level[ppvu_arg_inf[:,0]]/(pvu_sup-pvu_inf))*(pvu_sup-pvlev)
        
        for k in range (0,len(ppvu_arg_sup)):
            u_pvu[ppvu_arg_sup[k,1],ppvu_arg_sup[k,2]]=u1[ppvu_arg_sup[k,0],ppvu_arg_sup[k,1],ppvu_arg_sup[k,2]]*(pvlev-pvu_inf[k])/(pvu_sup[k]-pvu_inf[k])+(u1[ppvu_arg_inf[k,0],ppvu_arg_inf[k,1],ppvu_arg_inf[k,2]])*(pvu_sup[k]-pvlev)/(pvu_sup[k]-pvu_inf[k])
            v_pvu[ppvu_arg_sup[k,1],ppvu_arg_sup[k,2]]=v1[ppvu_arg_sup[k,0],ppvu_arg_sup[k,1],ppvu_arg_sup[k,2]]*(pvlev-pvu_inf[k])/(pvu_sup[k]-pvu_inf[k])+(v1[ppvu_arg_inf[k,0],ppvu_arg_inf[k,1],ppvu_arg_inf[k,2]])*(pvu_sup[k]-pvlev)/(pvu_sup[k]-pvu_inf[k])
        a=np.where(ppvu==0)
        
        ppvu[ppvu==0] =np.min(lev)
        for i in range(0,len(lat)):
            for j in range(0,len(lon)):
                if u_pvu[i][j]==0:
                    u_pvu[i][j]=u1[0,i,j]
        for i in range(0,len(lat)):
            for j in range(0,len(lon)):
                if v_pvu[i][j]==0:
                    v_pvu[i][j]=v1[0,i,j]
        
        ff_pvu=((u_pvu**2+v_pvu**2)**0.5)
        altpvu =altpvu+[-8300.*np.log(ppvu/1013.25)]
        u_pvu=u_pvu+[u_pvu]
        v_pvu=v_pvu+[v_pvu]
        ff_pvu=ff_pvu+[ff_pvu]
    altpvu=np.array(altpvu)
    u_pvu=np.array(u_pvu)
    v_pvu=np.array(v_pvu)
    ff_pvu=np.array(ff_pvu)
    
    altpvu=gaussian_filter(altpvu,sigma=3)
    u_pvu=gaussian_filter(u_pvu,sigma=3)
    v_pvu=gaussian_filter(v_pvu,sigma=3)
    ff_pvu=gaussian_filter(ff_pvu,sigma=3)
    
    print(altpvu)
    print(z_pvu)

    path=(dir_figs+storm+'/Z15PVU_Thetae850_vort925/')                          
    if not os.path.exists(path):
        os.mkdir(path)
    
    for l in tqdm(range(len(time))):
        #print(str(time[l])[0:13])
        fig = plt.figure(figsize=(17., 12.))
        ax = fig.add_subplot(1, 1, 1, projection=projection)
        ax.set_title('Storm '+storm+' - Z 1.5PVU, '+r"$\theta_e$ "+'at 850 hPa and relative vorticity at 925 hPa',loc='left',fontsize=14)
        ax.set_title(str(time[l])[0:13],loc='right',fontsize=14)
        
        plot_background(ax)
        ax.set_extent(*bounds, crs=ccrs.PlateCarree())
        boundary_path = make_boundary_path(lon, lat)
        ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())  
        
        c1 = ax.contour(lon, lat, altpvu[l,:,:], levels=z15_levels, colors="brown", linewidths=1, transform=ccrs.PlateCarree())
        ax.clabel(c1,fmt='%5.1i',fontsize=10)
        
        c3 = ax.contour(lon, lat, vort[l,:,:]*1e04, levels=vort_levels[vort_levels != 0], colors="black", linewidths=1, transform=ccrs.PlateCarree())
        ax.clabel(c3,fmt='%2.1i',fontsize=10)
        
        cf = ax.contourf(lon, lat, thetae[l,:,:], levels=th_levels, extend='both', cmap='jet', transform=ccrs.PlateCarree())
        cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
        cb.set_label('°C', size='large')
        
        #plt.show()
        
        figname=path+'Z15PVU_Thetae850_vort925_'+str(time[l])[0:13]
        fig.savefig(figname+'.png',bbox_inches='tight')
        plt.close()
        
    gif_filepath = dir_anim+storm+'/Z15PVU_Thetae850_vort925.gif'
    make_animation(path, gif_filepath)